<a href="https://colab.research.google.com/github/AndromedaOMA/Advanced_Analytics_with_Apache_Spark---E.On_Software_Development/blob/main/Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descriere proiect

Acest proiect are ca scop principal analiza consumului de energie al unui grup de consumatori fictivi, pe parcursul unui an, a unei companii de energie, folosind tehnicile de analiză din motorul Apache Spark.

1. Primul set de date reflectă atât consumul total de energie, cât și detalii specifice, dacă există, despre producția din panouri solare, consumul pentru vehicule electrice (EV), energia furnizată înapoi către rețeaua electrică, consumul și încărcarea bateriilor.
2. Al doilea set de date oferă atât tariful pe an și prețurile per kWh în diferite intervale de timp, unice pentru fiecare client în parte, cât și prețul de vânzare la nivelul companiei de energie pe diferite intervale de timp.

Proiectul implică curățarea și prelucrarea datelor, completarea valorilor lipsă, iar la final calculul facturii de energie, bonus fiind compararea facturii cu alți clienți similari.



---

# Seturile de date

## Pregătire mediu de lucru

In [1]:
from google.colab import drive
# drive.mount('/content/drive')
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [2]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://dlcdn.apache.org/spark/
!wget -q https://dlcdn.apache.org/spark/spark-3.4.4/spark-3.4.4-bin-hadoop3.tgz
!tar xf spark-3.4.4-bin-hadoop3.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j
import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.4.4-bin-hadoop3"
import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as f
spark= SparkSession.builder.getOrCreate()
spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:5 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,155 kB]
Get:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease [24.3 kB]
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,543 kB]
Get:13 http://security.ubuntu.com/ubuntu jammy-security/ma

## Setul de Date – Raw Time Series

Acest set de date conține informații despre consumul de date a unor clienți fictivi ai unei companii de energie. Structura datelor este una similară cu schemele folosite la momentul actual pentru astfel de date.

In [3]:
parquet_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/raw_time_series/parquet'
raw_time_series_df = spark.read.parquet(parquet_path)
raw_time_series_df.show(truncate=False)

+-------------------+-------------------+--------------------+------------+--------------------------+
|contract_id        |timestamp          |value               |value_source|annotations               |
+-------------------+-------------------+--------------------+------------+--------------------------+
|04_02_111 _ CHR12  |2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04 _02_111 _CHR12  |2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111 _ CHR12  |2023-01-01 17:30:22|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04 _02_111 _CHR12  |2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04 _ 02 _111_CHR12 |2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04 _02_111 _ CHR12 |2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04 _02 _111 _CHR12 |2023-01-03 10:45:00|0.931575            |measurement

## Setul de Date – Customer Tariff

Acest set de date conține informații despre tarifele și prețurile a unor clienți fictivi, într-un interval de timp, ai unei companii de energie. Structura este una similară cu cea folosite la momentul actual pentru astfel de date.

In [4]:
parquet_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/customer_tariff/parquet'
customer_tariff_df = spark.read.parquet(parquet_path)
customer_tariff_df.show()

+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|    contract_id|target_local_start_timestamp|target_local_end_timestamp|      tariff_name|charge_type| price|
+---------------+----------------------------+--------------------------+-----------------+-----------+------+
|04_02_111_CHR28|         2022-12-01 00:00:00|       2023-02-16 00:00:00|     Electric Pro|        buy| 0.302|
|04_02_111_CHR28|         2023-02-16 00:00:00|       2024-05-19 00:00:00| Electric Loyalty|        buy|0.3938|
|04_02_111_CHR28|         2024-05-19 00:00:00|       2024-08-31 00:00:00|     Eco Electric|        buy|0.2095|
|04_02_111_CHR28|         2024-08-31 00:00:00|       2024-10-09 00:00:00| Electric Loyalty|        buy|0.4047|
|04_02_111_CHR28|         2024-10-09 00:00:00|       2024-11-09 00:00:00|Business Electric|        buy|0.1716|
|04_02_111_CHR28|         2024-11-09 00:00:00|       2024-12-11 00:00:00|     Electric Pro|        buy|0.2944|
|

---

# Cerințe

## Curățarea Datelor

Setul de date al consumului prezintă unele probleme, precum spații suplimentare și informații lipsă sau semi-structurate.

Acestea trebuie rectificate înainte de a trece mai departe:

1. Curățarea coloanei „contract_id” de spații suplimentare
2. Setarea coloanei „value_source” cu valoarea „missing” atunci când valoarea lipsește.
3. Coloana „timestamp” are variații mici ce trebuie rectificate. Contoarele măsoară consumul odată la 15 minute exact începând cu ora 00:00, dar din cauza procesării, pot apărea variații la timpul pe care îl trimit. (Indiciu: Se găsește cea mai apropriat multiplu de 15 la minute și se scot secundele)



In [5]:
# 1.
raw_time_series_df = raw_time_series_df.withColumn('contract_id', f.translate(f.col('contract_id'), ' ', ''))
raw_time_series_df.show(truncate=False)

+---------------+-------------------+--------------------+------------+--------------------------+
|contract_id    |timestamp          |value               |value_source|annotations               |
+---------------+-------------------+--------------------+------------+--------------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:30:22|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-03 10:45:00|0.931575            |measurement |{"region":"Europe/Berlin"}|
|04_02_111

In [6]:
# 2.
# raw_time_series_df = raw_time_series_df.withColumn('value_source', f.coalesce(f.col('value_source'), f.lit('missing')))
raw_time_series_df = raw_time_series_df.withColumn('value_source', f.when(f.col('value').isNull(), f.lit('missing')).otherwise(f.col('value_source')))
raw_time_series_df.show(truncate=False)

+---------------+-------------------+--------------------+------------+--------------------------+
|contract_id    |timestamp          |value               |value_source|annotations               |
+---------------+-------------------+--------------------+------------+--------------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:30:22|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-03 10:45:00|0.931575            |measurement |{"region":"Europe/Berlin"}|
|04_02_111

In [7]:
# 3.
raw_time_series_df = raw_time_series_df.withColumn('timestamp',
                                                   f.from_unixtime((f.unix_timestamp('timestamp') / 900).cast('int') * 900).cast('timestamp'))
raw_time_series_df.show(truncate=False)

+---------------+-------------------+--------------------+------------+--------------------------+
|contract_id    |timestamp          |value               |value_source|annotations               |
+---------------+-------------------+--------------------+------------+--------------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-02 05:30:00|0.03638379308965683 |measurement |{"region":"Europe/Berlin"}|
|04_02_111_CHR12|2023-01-03 10:45:00|0.931575            |measurement |{"region":"Europe/Berlin"}|
|04_02_111

## Extragerea Informațiilor de localizare și filtrarea datelor invalide

Setul de date al consumului prezintă unele probleme, precum spații suplimentare și informații lipsă sau semi-structurate.

Acestea trebuie rectificate înainte de a trece mai departe:

1. Extragerea unei noi coloane „region” din „annotations” (Se recomandă folosirea funcțiilor de Spark de procesare JSON)
2. Clienții cu regiuni invalide se vor scoate din setul de date și se vor salva pe disk într-o locație separată.
3. Extragerea datei din coloana „timestamp” într-o nouă coloană „utc_date”
4. Calcularea datei locale pentru data și ora din „timestamp”, pe baza regiunii, într-o nouă coloană „local_timestamp”
5. Extragerea datei din coloana „ local_timestamp” într-o nouă coloană „local_date”



In [8]:
# 1.
raw_time_series_df = raw_time_series_df.withColumn('region', f.get_json_object(f.col('annotations'), '$.region'))
print(raw_time_series_df.count())
raw_time_series_df.show(5, truncate=False)

3549244
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
+---------------+-------------------+-------------------+------------+--------------------------

In [9]:
# 2.
# Am verificat câte regiuni se găsesc în setul nostru de date -> REZULTAT: Europe/Berlin și WakaWaka
view_regions = raw_time_series_df.select('region').distinct().collect()
print(view_regions)

test_raw_time_series_df = raw_time_series_df.groupBy('region').avg()
test_raw_time_series_df.show(truncate=False)

# Așadar:
out_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/outputs/invalid_regions'
invalid_regions_df = raw_time_series_df.where(f.col('region')!='Europe/Berlin')
invalid_regions_df.write.mode('overwrite').format('parquet').save(out_path)

# print(f"no. of rows of raw_time_series_df: {raw_time_series_df.count()}")
raw_time_series_df.show(5, truncate=False)

# print(f"no. of rows of invalid_regions_df: {invalid_regions_df.count()}")
invalid_regions_df.show(5, truncate=False)

[Row(region='Europe/Berlin'), Row(region='WakaWaka')]
+-------------+-------------------+
|region       |avg(value)         |
+-------------+-------------------+
|Europe/Berlin|0.490498974431521  |
|WakaWaka     |0.13178303122889254|
+-------------+-------------------+

+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|

In [10]:
# 3.
raw_time_series_df = raw_time_series_df.withColumn('utc_date', f.to_date(f.col('timestamp')))
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|
|04_02_111_CHR12|2023-01-02 00:30:00|0.03597601881331959|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-02|
+---------------

In [11]:
# 4.
raw_time_series_df = raw_time_series_df.filter(f.col('region') == 'Europe/Berlin')
raw_time_series_df = raw_time_series_df.withColumn('local_timestamp', f.from_utc_timestamp(f.col('timestamp'), f.col('region')))
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:00:00|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:30:00|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 22:30:00|

In [12]:
# 5.
raw_time_series_df = raw_time_series_df.withColumn('local_date', f.to_date('local_timestamp'))
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:00:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:30:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 21:30:00|0.08670661371854547|measurement |{"regi

## Extragerea Informațiilor de consum

Setul de date al consumului prezintă unele probleme, precum spații suplimentare și informații lipsă sau semi-structurate.



Acestea trebuie rectificate înainte de a trece mai departe:

1. Extragerea din coloana „annotations” a consumului de vehicul electric (EV), baterie (BATTERY_IN) și consumul trimis spre rețeaua electrică (GRID_SELL) în coloanele „sent_to_ev”, „sent_to_battery” și „sent_to_grid”. În cazul în care valoare lipsește, se consideră consumul 0.
2. Extragerea din coloana „annotations” a energiei primite de la panourile solare (PV) și baterie (BATTERY_OUT) în coloanele „received_from_pv” și „received_from_battery”. În cazul în care valoare lipsește, se consideră energia primită 0.

In [13]:
print("Seturile de date disponibile:")
print("raw_time_series_df")
raw_time_series_df.show(5, truncate=False)
print("customer_tariff_df")
customer_tariff_df.show(5, truncate=False)

Seturile de date disponibile:
raw_time_series_df
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:00:00|2023-01-01|
|04_02_111_CHR12|2023-01-01 17:30:00|0.08180149515221906|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 18:30:00|2023-01-01|
|04_02_111_CHR12|2023-01-01

In [14]:
# 1.

# Verificăm tipul de date a coloanei "annotations":
print("Tipurile valorilot coloanei annotations:")
raw_time_series_df.select('annotations').printSchema()
# Verificăm valorile distincte pe care le primește coloana "annotations":
print("Valorile distincte ale coloanei annotations:")
raw_time_series_df.select(('annotations')).distinct().show(5, truncate=False)
print('===' * 29)

raw_time_series_df = raw_time_series_df.withColumn('sent_to_ev', f.when(f.get_json_object(f.col('annotations'), '$.events.EV').isNotNull(), f.get_json_object(f.col('annotations'), '$.events.EV')).otherwise(0))
raw_time_series_df = raw_time_series_df.withColumn('sent_to_battery', f.when(f.get_json_object(f.col('annotations'), '$.events.BATTERY_IN').isNotNull(), f.get_json_object(f.col('annotations'), '$.events.BATTERY_IN')).otherwise(0))
raw_time_series_df = raw_time_series_df.withColumn('sent_to_grid', f.when(f.get_json_object(f.col('annotations'), '$.events.GRID_SELL').isNotNull(), f.get_json_object(f.col('annotations'), '$.events.GRID_SELL')).otherwise(0))
raw_time_series_df.show(5,truncate=False)

# Pentru validarea faptului că valorile consumurilor nu sunt numai nule
raw_time_series_df.select('sent_to_ev').distinct().show(5,truncate=False)
raw_time_series_df.select('sent_to_battery').distinct().show(5,truncate=False)
raw_time_series_df.select('sent_to_grid').distinct().show(5,truncate=False)


Tipurile valorilot coloanei annotations:
root
 |-- annotations: string (nullable = true)

Valorile distincte ale coloanei annotations:
+-------------------------------------------------------------------------------------+
|annotations                                                                          |
+-------------------------------------------------------------------------------------+
|{"region":"Europe/Berlin","events":{"GRID_SELL":"1.753088873001001","PV":"1.839175"}}|
|{"region":"Europe/Berlin","events":{"EV":"1.035","PV":"1.674475"}}                   |
|{"region":"Europe/Berlin","events":{"PV":"1.076"}}                                   |
|{"region":"Europe/Berlin","events":{"PV":"1.077475"}}                                |
|{"region":"Europe/Berlin","events":{"BATTERY_IN":"1.633989566467353","PV":"1.69435"}}|
+-------------------------------------------------------------------------------------+
only showing top 5 rows

+---------------+-------------------+-----------

In [15]:
# 2.
raw_time_series_df = raw_time_series_df.withColumn('received_from_pv', f.when(f.get_json_object(f.col('annotations'), '$.events.PV').isNotNull(), f.get_json_object(f.col('annotations'), '$.events.PV')).otherwise(0))
raw_time_series_df = raw_time_series_df.withColumn('received_from_battery', f.when(f.get_json_object(f.col('annotations'), '$.events.BATTERY_OUT').isNotNull(), f.get_json_object(f.col('annotations'), '$.events.PV')).otherwise(0))

raw_time_series_df.show(5,truncate=False)
raw_time_series_df.select('received_from_pv').distinct().show(5,truncate=False)
raw_time_series_df.select('received_from_battery').distinct().show(5,truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|2023-01-01|0         |0              |0           |0               |0                    |
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement

## Filtrarea consumului neobișnuit

Anumite valori ale consumului sunt neobișnuit de mari și este necesară scoaterea lor.



1.  Setarea coloanei „value_source” în „plausability_check_failed” pentru valorile cu consum neobișnuit din setul de date. (Decizia valorilor mari fie se face cu o analiză vizuală a datelor (sortarea și identificarea lor vizual fie prin agregări simple fie mai complicate) sau bonus, pentru cine dorește, prin tehnici de învățare automată.)
2. Salvarea separată a datelor cu „value_source” având valoare „plausability_check_failed” într-o locație separată. Atenție, datele nu se scot din setul de date.
3. Setarea coloanei „value” în NULL pentru datele cu „value_source” având valoare „plausability_check_failed”.



In [16]:
raw_time_series_df.show(5, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Berlin"}|Europe/Berlin|2023-01-01|2023-01-01 07:00:00|2023-01-01|0         |0              |0           |0               |0                    |
|04_02_111_CHR12|2023-01-01 17:00:00|0.07385444264936832|measurement

In [17]:
# 1. IMPLEMENTAREA I

# Prea multă putere de calcul necesară pentru realizarea plot-urilor
# pandas_df = view_details_rts_df.toPandas()
# pandas_df.value.value_counts().plot(kind='bar')

view_value_sources = raw_time_series_df.select('value').distinct().orderBy(f.col('value').desc()).limit(2000)
print("view_value_sources: ")
view_value_sources.show(5, truncate=False)
out_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/outputs/view_details_rts_df'
view_value_sources.coalesce(1).write.mode('overwrite').csv(out_path)

# Din CSV-ul salvat mai sus deducem aspectul constant a creșterii valorilor, dar în intervalul 39-42 valorile încep să crească mai alarmant.
raw_time_series_df = raw_time_series_df.withColumn('value_source', f.when(f.col('value')>39, 'plausability_check_failed').otherwise(f.col('value_source')))
print("raw_time_series_df: ")
raw_time_series_df.show(5, truncate=False)


view_value_sources: 
+-----------------+
|value            |
+-----------------+
|83.7633960367971 |
|83.45165458104191|
|80.5203908045932 |
|77.88889162124525|
|77.48032877642238|
|76.88911895599986|
|76.69798782063941|
|75.76717163660166|
|75.63363148646712|
|75.1682792514436 |
|74.9268292510051 |
|74.39430262643208|
|74.17691963148631|
|73.79597194030637|
|73.62673236058076|
|73.50758279260636|
|73.24851498214987|
|73.1949246302417 |
|72.90555560294808|
|72.73071224721161|
+-----------------+
only showing top 20 rows

raw_time_series_df: 
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_

In [18]:
# 1. IMPLEMENTAREA II (BONUS)
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.clustering import KMeans

# Corectare coloană 'value' prin ștergerea valorilor de 0
raw_time_series_df = raw_time_series_df.where(f.col('value')!=0)

# Vectorizare pe coloana 'value'
vector_assembler = VectorAssembler(inputCols=['value'], outputCol='value_features')
data_vectorized_df = vector_assembler.transform(raw_time_series_df)

# KMeans
kmeans = KMeans(featuresCol='value_features', predictionCol='cluster', k=2)
model = kmeans.fit(data_vectorized_df)

# Calculează distanța față de centrul clusterului
from pyspark.sql.types import DoubleType

center = model.clusterCenters()[0]

def distance(v):
    return float((v - center).norm(2))

distance_udf = f.udf(distance, DoubleType())
data_with_distance = data_vectorized_df.withColumn("distance", distance_udf("value_features"))

# Setează „value” pentru valori anormale
threshold = 3.0
raw_time_series_df = data_with_distance.withColumn(
    "value_source",
    f.when(f.col("distance") > threshold, "plausability_check_failed")
    .otherwise(f.col("value_source"))
)

view_value_sources = raw_time_series_df.select('value_source').distinct()
print("view_value_sources: ")
view_value_sources.show(truncate=False)
print("raw_time_series_df: ")
raw_time_series_df.show(5, truncate=False)

view_value_sources: 
+-------------------------+
|value_source             |
+-------------------------+
|measurement              |
|plausability_check_failed|
+-------------------------+

raw_time_series_df: 
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+---------------------+-------------------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|value_features       |distance           |
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+-------

In [19]:
# 2.
out_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/outputs/plausability_check_failed'
raw_time_series_df.coalesce(1).where(f.col('value_source')=='plausability_check_failed').write.mode('overwrite').format('parquet').save(out_path)

In [20]:
# 3.
raw_time_series_df = raw_time_series_df.withColumn('value', f.when(f.col('value_source') == 'plausability_check_failed', f.lit('Null')).otherwise(f.col('value')))
view_value = raw_time_series_df.select('value').distinct()
view_value.show(5, truncate=False)

+-------------------+
|value              |
+-------------------+
|0.02906966450736488|
|0.7859             |
|0.1538957677168379 |
|0.2442938179085887 |
|0.8454496624316684 |
+-------------------+
only showing top 5 rows



## Completarea valorilor lipsă

Anumiți clienți au lipsuri prezintă câteva lipsuri în consum, unele dintre ele adăugate de noi la pasul precedent:




1. Prezicerea valorii „value” atunci când ea este NULL folosind următoarea metodă:

  * Calculăm mai întâi media din ultimele 8 săptămâni a valorilor din aceeași zi a săptămânii la aceeași oră, minut și secundă. Se folosesc din ultimele 8 săptămâni doar valorile care nu lipsesc, au coloana „value_source” setată pe valoarea „measurement”.
  * Facem suma coloanelor „sent_to_ev”, „sent_to_battery” și „sent_to_grid”.
  * Facem suma coloanelor „received_from_pv” și „received_from_battery”.
  * Completăm coloane „value” cu maximum dintre aceste 3 valori.
  * **Bonus**, pentru cine dorește, puteți folosi și algoritmi de învățare automată, precum Linear Regression, în loc de calcularea mediei din ultimele 8 săptămâni și să comparați cele 2 metode.


2. Calcularea valorii „received_from_grid” atunci când avem toate informațiile.
  * Facem suma coloanelor „sent_to_ev”, „sent_to_battery” și „sent_to_grid” și scădem valorile din coloanele „received_from_pv” și „received_from_battery”.



In [21]:
# Verificăm existența valorilor Null
null_values = raw_time_series_df.select(f.when(f.col('value').isNull() & (f.col("value_source") == 'measurement'), f.lit('Null')).alias('null_values'))
null_values.show(5, truncate=False)

+-----------+
|null_values|
+-----------+
|NULL       |
|NULL       |
|NULL       |
|NULL       |
|NULL       |
+-----------+
only showing top 5 rows



In [22]:
# IMPLEMENTAREA I

# Calculăm mai întâi media din ultimele 8 săptămâni a valorilor din aceeași zi a săptămânii la aceeași oră, minut și secundă. Se folosesc din ultimele 8 săptămâni doar valorile care nu lipsesc, au coloana „value_source” setată pe valoarea „measurement”
raw_time_series_df = (raw_time_series_df.withColumn('week_day', f.dayofweek(f.col('timestamp')))
                                        .withColumn('hour', f.hour(f.col('timestamp')))
                                        .withColumn('minute', f.minute(f.col('timestamp')))
                                        .withColumn('second', f.second(f.col('timestamp'))))
raw_time_series_df.show(5, truncate=False)

now = f.current_timestamp()
eight_weeks_ago = f.date_sub(now, 56)

filtered_df = raw_time_series_df.filter(
    (f.col('timestamp') >= eight_weeks_ago).isNotNull() &
    f.col('value').isNotNull() &
    (f.col('value_source') == 'measurement').isNotNull()
)
filtered_df.show(5, truncate=False)

mean_df = filtered_df.groupBy("week_day", "hour", "minute", "second") \
    .agg(f.avg("value").alias("mean_value"))

mean_df.show(10, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+---------------------+-------------------+--------+----+------+------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|value_features       |distance           |week_day|hour|minute|second|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+---------------------+-------------------+--------+----+------+------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Ber

In [23]:
raw_time_series_df.select((f.col('value_source') == 'measurement') & (f.col('value_source').isNotNull())).show(4, truncate=False)

+-------------------------------------------------------------+
|((value_source = measurement) AND (value_source IS NOT NULL))|
+-------------------------------------------------------------+
|true                                                         |
|true                                                         |
|true                                                         |
|true                                                         |
+-------------------------------------------------------------+
only showing top 4 rows



In [24]:
raw_time_series_df.select(f.col('value').isNotNull()).show(4, truncate=False)

+-------------------+
|(value IS NOT NULL)|
+-------------------+
|true               |
|true               |
|true               |
|true               |
+-------------------+
only showing top 4 rows



In [25]:
filter1 = raw_time_series_df.filter((f.col('value_source') == 'measurement').isNotNull())
filter1.show(4, truncate=False)

+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+---------------------+-------------------+--------+----+------+------+
|contract_id    |timestamp          |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|value_features       |distance           |week_day|hour|minute|second|
+---------------+-------------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+---------------------+-------------------+--------+----+------+------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.02591860654732236|measurement |{"region":"Europe/Ber

In [26]:
# salvăm setul de date
# out_path = '/content/drive/MyDrive/E.on/E.on_Data/Data/Project/outputs/raw_time_series_df'
# raw_time_series_df.coalesce(1).write.mode('overwrite').format('parquet').save(out_path)

In [27]:
test_raw_time_series_df = raw_time_series_df
test_raw_time_series_df.select('value_source').distinct().show()
test_raw_time_series_df.filter(test_raw_time_series_df.value.isNotNull()).count()
test_raw_time_series_df.select(f.min('timestamp'), f.max('timestamp')).show()

+--------------------+
|        value_source|
+--------------------+
|         measurement|
|plausability_chec...|
+--------------------+

+-------------------+-------------------+
|     min(timestamp)|     max(timestamp)|
+-------------------+-------------------+
|2023-01-01 00:45:00|2024-01-01 00:45:00|
+-------------------+-------------------+



In [28]:

# Facem suma coloanelor „sent_to_ev”, „sent_to_battery” și „sent_to_grid”.
raw_time_series_df = raw_time_series_df.withColumn(
    'sum_sent',
    f.col('sent_to_ev') + f.col('sent_to_battery') + f.col('sent_to_grid')
)

# Facem suma coloanelor „received_from_pv” și „received_from_battery”.
raw_time_series_df = raw_time_series_df.withColumn(
    'sum_received',
    f.col('received_from_pv') + f.col('received_from_battery')
)

raw_time_series_df.show(5)

+---------------+-------------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+--------------------+-------------------+--------+----+------+------+--------+------------+
|    contract_id|          timestamp|              value|value_source|         annotations|       region|  utc_date|    local_timestamp|local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|      value_features|           distance|week_day|hour|minute|second|sum_sent|sum_received|
+---------------+-------------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+--------------------+-------------------+--------+----+------+------+--------+------------+
|04_02_111_CHR12|2023-01-01 06:00:00|0.0259186

In [29]:

# Completăm coloane „value” cu maximum dintre aceste 3 valori.

# Join pe zi, oră, minut, secundă
raw_time_series_df = raw_time_series_df.join(
    mean_df,
    on=['week_day', 'hour', 'minute', 'second'],
    how='left'
)

raw_time_series_df.show(5)

+--------+----+------+------+---------------+-------------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+--------------------+-------------------+--------+------------+-------------------+
|week_day|hour|minute|second|    contract_id|          timestamp|              value|value_source|         annotations|       region|  utc_date|    local_timestamp|local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|      value_features|           distance|sum_sent|sum_received|         mean_value|
+--------+----+------+------+---------------+-------------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+--------------------+-------------------+--------+------------+-------

In [30]:

# Alegem maximum dintre mean_value, sum_sent și sum_received
value_candidate = f.greatest(f.col('`mean_value`'), f.col('sum_sent'), f.col('sum_received'))

# Înlocuim valoarea doar dacă value este NULL
raw_time_series_df = raw_time_series_df.withColumn(
    'value',
    f.when(f.col('value').isNull(), value_candidate).otherwise(f.col('value'))
)

raw_time_series_df.show(5)

+--------+----+------+------+---------------+-------------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+--------------------+-------------------+--------+------------+-------------------+
|week_day|hour|minute|second|    contract_id|          timestamp|              value|value_source|         annotations|       region|  utc_date|    local_timestamp|local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|      value_features|           distance|sum_sent|sum_received|         mean_value|
+--------+----+------+------+---------------+-------------------+-------------------+------------+--------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+--------------------+-------------------+--------+------------+-------

In [31]:
# IMPLEMENTAREA II

from pyspark.ml.feature import VectorAssembler
from pyspark.ml.regression import LinearRegression

# Pregătim trăsături (features) — le ai deja create
features = ['week_day', 'hour', 'minute', 'second', 'sum_sent', 'sum_received']

# Adăugăm eticheta 'label' doar dacă sursa este 'measurement' și valoarea nu este NULL
ml_df = raw_time_series_df.withColumn(
    'label',
    f.when(
        (f.col('value_source') == 'measurement') & f.col('value').isNotNull(),
        f.col('value').cast(DoubleType())
    )
)

# Filtrăm doar rândurile unde avem etichetă validă
train_df = ml_df.filter(f.col('label').isNotNull())

# Asamblăm vectorul de trăsături
assembler = VectorAssembler(inputCols=features, outputCol='features')
train_df = assembler.transform(train_df)

# Antrenăm modelul de regresie
lr = LinearRegression(featuresCol='features', labelCol='label')
lr_model = lr.fit(train_df)


In [32]:
# Pregătim rândurile fără valoare pentru predicție
predict_df = ml_df.filter(f.col('value').isNull())
predict_df = assembler.transform(predict_df)

# Facem predicția
predicted_df = lr_model.transform(predict_df).select('timestamp', 'prediction')

# Adăugăm predicția înapoi în dataframe-ul original
raw_time_series_df = raw_time_series_df.join(predicted_df, on='timestamp', how='left')

# Completăm coloana `value` DOAR dacă e NULL cu predicția ML
raw_time_series_df = raw_time_series_df.withColumn(
    'value_lr',
    f.when(f.col('value').isNull(), f.col('prediction')).otherwise(f.col('value'))
)


In [33]:
# 2.
raw_time_series_df = raw_time_series_df.withColumn('received_from_grid', f.col('sent_to_ev') + f.col('sent_to_battery') + f.col('sent_to_grid') - f.col('received_from_pv') - f.col('received_from_battery'))
raw_time_series_df.show(5, truncate=False)

+-------------------+--------+----+------+------+---------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+------------+----------------+---------------------+---------------------+-------------------+--------+------------+-------------------+----------+-------------------+------------------+
|timestamp          |week_day|hour|minute|second|contract_id    |value              |value_source|annotations               |region       |utc_date  |local_timestamp    |local_date|sent_to_ev|sent_to_battery|sent_to_grid|received_from_pv|received_from_battery|value_features       |distance           |sum_sent|sum_received|mean_value         |prediction|value_lr           |received_from_grid|
+-------------------+--------+----+------+------+---------------+-------------------+------------+--------------------------+-------------+----------+-------------------+----------+----------+---------------+--

## Asocierea cu tarifere

Curățând datele și completând valorile lipsă, putem acuma să trecem la asocierea cu tarifele consumatorilor:



1. Asocierea intrărilor de consum cu cele de tarifare, pe bază numărului de contract, al timpului și al tipului de preț, cumpărare sau vânzare.
  * Prețul tarifat la cumpărare este prețul de cumpărare pentru intrare de consum înmulțit cu coloana „received_from_grid”
  * Prețul tarifat la vânzare este prețul de vânzare pentru intrare de consum înmulțit cu coloana „send_to_grid”.

In [142]:
# print(f'raw_time_series_df: {raw_time_series_df.schema} \n {raw_time_series_df.show(5)}')
# print(f'customer_tariff_df: {customer_tariff_df.schema} \n {customer_tariff_df.show(5)}')

print(f'raw_time_series_df: \n{raw_time_series_df.schema}')
print(f'customer_tariff_df: \n{customer_tariff_df.schema}')

raw_time_series_df: 
StructType([StructField('timestamp', TimestampType(), True), StructField('week_day', IntegerType(), True), StructField('hour', IntegerType(), True), StructField('minute', IntegerType(), True), StructField('second', IntegerType(), True), StructField('contract_id', StringType(), True), StructField('value', StringType(), True), StructField('value_source', StringType(), True), StructField('annotations', StringType(), True), StructField('region', StringType(), True), StructField('utc_date', DateType(), True), StructField('local_timestamp', TimestampType(), True), StructField('local_date', DateType(), True), StructField('sent_to_ev', StringType(), True), StructField('sent_to_battery', StringType(), True), StructField('sent_to_grid', StringType(), True), StructField('received_from_pv', StringType(), True), StructField('received_from_battery', StringType(), True), StructField('value_features', VectorUDT(), True), StructField('distance', DoubleType(), True), StructField('su

In [143]:
raw_df = raw_time_series_df.withColumn('local_timestamp', f.to_timestamp('local_timestamp'))
print(f'raw_df->schema: \n{raw_df.schema}')

tariff_df = customer_tariff_df \
    .withColumn('target_local_start_timestamp', f.to_timestamp('target_local_start_timestamp')) \
    .withColumn('target_local_end_timestamp', f.to_timestamp('target_local_end_timestamp'))
print(f'tariff_df->schema: \n{tariff_df.schema}')

raw_df->schema: 
StructType([StructField('timestamp', TimestampType(), True), StructField('week_day', IntegerType(), True), StructField('hour', IntegerType(), True), StructField('minute', IntegerType(), True), StructField('second', IntegerType(), True), StructField('contract_id', StringType(), True), StructField('value', StringType(), True), StructField('value_source', StringType(), True), StructField('annotations', StringType(), True), StructField('region', StringType(), True), StructField('utc_date', DateType(), True), StructField('local_timestamp', TimestampType(), True), StructField('local_date', DateType(), True), StructField('sent_to_ev', StringType(), True), StructField('sent_to_battery', StringType(), True), StructField('sent_to_grid', StringType(), True), StructField('received_from_pv', StringType(), True), StructField('received_from_battery', StringType(), True), StructField('value_features', VectorUDT(), True), StructField('distance', DoubleType(), True), StructField('sum_se

In [144]:
# Join pentru tariful de cumpărare
buy_tariff = tariff_df.filter(f.col('charge_type') == 'buy') \
    .withColumnRenamed('contract_id', 'buy_contract_id')
print(f'buy_tariff->schema: \n{buy_tariff.schema}')

raw_with_buy = raw_df.join(
    buy_tariff,
    (raw_df['contract_id'] == buy_tariff['buy_contract_id']) &
    (raw_df['local_timestamp'] >= buy_tariff['target_local_start_timestamp']) &
    (raw_df['local_timestamp'] < buy_tariff['target_local_end_timestamp']),
    how='left'
).withColumn('buy_cost', f.col('price') * f.col('received_from_grid'))

print(f'raw_with_buy->schema: \n{raw_with_buy.schema}')

buy_tariff->schema: 
StructType([StructField('buy_contract_id', StringType(), True), StructField('target_local_start_timestamp', TimestampType(), True), StructField('target_local_end_timestamp', TimestampType(), True), StructField('tariff_name', StringType(), True), StructField('charge_type', StringType(), True), StructField('price', DoubleType(), True)])
raw_with_buy->schema: 
StructType([StructField('timestamp', TimestampType(), True), StructField('week_day', IntegerType(), True), StructField('hour', IntegerType(), True), StructField('minute', IntegerType(), True), StructField('second', IntegerType(), True), StructField('contract_id', StringType(), True), StructField('value', StringType(), True), StructField('value_source', StringType(), True), StructField('annotations', StringType(), True), StructField('region', StringType(), True), StructField('utc_date', DateType(), True), StructField('local_timestamp', TimestampType(), True), StructField('local_date', DateType(), True), StructFie

In [145]:
# Join pentru tariful de vânzare
sell_tariff = tariff_df.filter(f.col('charge_type') == 'sell') \
    .withColumnRenamed('contract_id', 'sell_contract_id')
print(f'sell_tariff->schema: \n{sell_tariff.schema}')

# Aplică aliasuri pentru DataFrame-uri
raw_with_buy_alias = raw_with_buy.alias('raw')
sell_tariff_alias = sell_tariff.alias('sell')

# Join pentru tariful de vânzare, folosind aliasuri și referințe explicite
raw_with_sell = raw_with_buy_alias.join(
    sell_tariff_alias,
    (f.col('raw.contract_id') == f.col('sell.sell_contract_id')) &
    (f.col('raw.local_timestamp') >= f.col('sell.target_local_start_timestamp')) &
    (f.col('raw.local_timestamp') < f.col('sell.target_local_end_timestamp')),
    how='left'
).withColumn('sell_cost', f.col('sell.price') * f.col('raw.sent_to_grid'))

print(f'raw_with_sell->schema: \n{raw_with_sell.schema}')



sell_tariff->schema: 
StructType([StructField('sell_contract_id', StringType(), True), StructField('target_local_start_timestamp', TimestampType(), True), StructField('target_local_end_timestamp', TimestampType(), True), StructField('tariff_name', StringType(), True), StructField('charge_type', StringType(), True), StructField('price', DoubleType(), True)])
raw_with_sell->schema: 
StructType([StructField('timestamp', TimestampType(), True), StructField('week_day', IntegerType(), True), StructField('hour', IntegerType(), True), StructField('minute', IntegerType(), True), StructField('second', IntegerType(), True), StructField('contract_id', StringType(), True), StructField('value', StringType(), True), StructField('value_source', StringType(), True), StructField('annotations', StringType(), True), StructField('region', StringType(), True), StructField('utc_date', DateType(), True), StructField('local_timestamp', TimestampType(), True), StructField('local_date', DateType(), True), Struct

In [146]:
# Selectarea coloanelor utile, folosind aliasuri pentru a evita ambiguitatea
raw_with_sell_selected = raw_with_sell.select(
    'timestamp',  # Timpul la care se înregistrează intrarea
    'week_day',  # Ziua săptămânii
    'hour',  # Ora
    'minute',  # Minutul
    'second',  # Secunda
    'contract_id',  # ID-ul contractului

    'value',  # Valoarea consumului
    'received_from_grid',  # Cantitatea primită din rețea
    'received_from_battery', # Cantitatea primită de la baterie
    'received_from_pv', #  Cantitatea primită de la pv
    'sent_to_ev',
    'buy_cost',
    'sent_to_grid',  # Cantitatea trimisă la rețea
    'sell_cost',  # Costul de vânzare
)

print(f'raw_with_sell_selected->schema: \n{raw_with_sell_selected.schema}')


raw_with_sell_selected->schema: 
StructType([StructField('timestamp', TimestampType(), True), StructField('week_day', IntegerType(), True), StructField('hour', IntegerType(), True), StructField('minute', IntegerType(), True), StructField('second', IntegerType(), True), StructField('contract_id', StringType(), True), StructField('value', StringType(), True), StructField('received_from_grid', DoubleType(), True), StructField('received_from_battery', StringType(), True), StructField('received_from_pv', StringType(), True), StructField('sent_to_ev', StringType(), True), StructField('buy_cost', DoubleType(), True), StructField('sent_to_grid', StringType(), True), StructField('sell_cost', DoubleType(), True)])


## Calcularea consumului și a facturii

După asociere, putem trece la calculul factorii:


1. Per zi / săptămâna / lună / an, pentru fiecare client, să se calculeze:
  * Consumul de energie total (suma coloanei value) – „ kWh_total”
  * Energia extrasă din baterie și cea din PV – „kWh_from_battery”, „kWh_from_PV”
  * Energia folosită pentru EV – „kWh_for_EV”
  * Consumul de energie folosit de la rețea – „kWh_from_grid”
  * Costul consumului de energie folosit de la rețea – „price_billed”
  * Consumul de energie trimis către rețea – „kWh_to_grid”
  * Costul primit înapoi, a consumului de energie trimis către rețea – „price_cashback”
  * Costul total (diferența de cost) – „price_final”
  * Puneți codul de calculare al agregatelor pe orice perioadă de timp într-o funcție, iar pentru fiecare interval, zi / săptămâna / lună / an, rulați această funcție. Afișați pentru fiecare perioada top 10 clienți cu cel mai mare preț.

In [ ]:
from pyspark.sql.window import Window

def calculate_aggregates(df, period_column):
    """
    Calculează agregatele pentru fiecare client pe baza unui interval de timp specificat.
    :param df: DataFrame-ul cu datele de consum și tarife
    :param period_column: Coloana care reprezintă perioada de agregare (zi, săptămână, lună, an)
    :return: DataFrame cu agregatele calculate
    """
    # Adăugarea coloanei 'local_date' dacă nu există (folosim timestamp pentru a crea data locală)
    df = df.withColumn('local_date', f.to_date(df['timestamp']))

    # Calcularea agregatelor
    aggregates = df.groupBy('contract_id', period_column).agg(
        f.sum('value').alias('kWh_total'),
        f.sum('received_from_battery').alias('kWh_from_battery'),
        f.sum('received_from_pv').alias('kWh_from_PV'),
        f.sum('sent_to_ev').alias('kWh_for_EV'),
        f.sum('received_from_grid').alias('kWh_from_grid'),
        f.sum('buy_cost').alias('price_billed'),
        f.sum('sent_to_grid').alias('kWh_to_grid'),
        f.sum('sell_cost').alias('price_cashback')
    )

    # Calcularea costului final
    aggregates = aggregates.withColumn('price_final', f.col('price_billed') - f.col('price_cashback'))

    # Crearea unui feronici pentru a ordona clienții după prețul final
    window_spec = Window.partitionBy(period_column).orderBy(f.col('price_final').desc())

    # Selectarea top 10 clienți pe fiecare perioadă
    top_10_customers = aggregates.withColumn('rank', f.row_number().over(window_spec)) \
                                .filter(f.col('rank') <= 10)

    return top_10_customers

# Perioada de timp - pe zi
top_10_daily = calculate_aggregates(raw_with_sell_selected, 'local_date')
top_10_daily.show(5)

# Perioada de timp - pe săptămână (folosind numărul săptămânii)
top_10_weekly = calculate_aggregates(raw_with_sell_selected, 'week_day')
top_10_weekly.show(5)

# Perioada de timp - pe lună (folosind luna și anul)
top_10_monthly = calculate_aggregates(raw_with_sell_selected, 'local_month')
top_10_monthly.show(5)

# Perioada de timp - pe an (folosind anul)
top_10_yearly = calculate_aggregates(raw_with_sell_selected, 'local_year')
top_10_yearly.show(5)
